<h1>Experiment on bilateral adaptive weights - Middlebury 2014 - sum</h1>
<h2>For results and discussion please visit&nbsp;<a href="ALG_006_EXP_005-Comparative_analysis-VIS.ipynb" target="_blank" rel="noopener">ALG_006_EXP_005-Comparative_analysis-VIS</a>.&nbsp;</h2>

In [1]:
import pandas as pd
import ipywidgets as widgets
import numpy as np
import sys
import os

sys.path.append(os.path.join("..", ".."))
import glob
import cv2

import plotly.graph_objs as go
import plotly.express as px
from ipywidgets import HBox, VBox, Button

from components.utils import plotly_helpers as ph
from components.utils import utils as u

In [2]:

available_metrix = ['abs_error',
       'mse', 'avg', 'eucledian', 'bad1', 'bad2', 'bad4', 'bad8']

metrics_selector = widgets.Dropdown(
    options=[(m,m) for m in available_metrix],
    description='Metric:',
    value="bad4"
)


nonoccluded = widgets.Dropdown(
    options=[("yes", False), ("No", True)],
    description='Nonoccluded:'
)


### Please select metrics and whether occlusions are counted as errors

In [3]:
VBox([metrics_selector, nonoccluded])

### Loading the data and building the plots

In [4]:

selected_file = os.path.join("..","..", "benchmarking", "MiddEval", "custom_log", "blg_sum_benchmarking.csv")
#selected_file = "./fixed_csv2.csv"
df = ph.load_n_clean(selected_file)

##Filtering to selected occlusion parameter

df = df[df["are_occlusions_errors"]==nonoccluded.value]
df.sort_values(by=["scene", "match", "h", "w"], inplace=True)

number_of_samples = df.shape[0]

In [5]:
### Dashboard 1


from ipywidgets import Image, Layout

img_widget = Image(value=df["loaded_imgs"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))

fig_a = ph.get_figure_widget (df, "scene", metrics_selector.value, 
                           "Scene w.r.t."+metrics_selector.value)
fig_b = ph.get_figure_widget (df, "match", "kernel_size", "Kernel sizes w.r.t. match values")


figs = [fig_a, fig_b]
ph.bind_hover_function(figs, img_widget, df)
ph.bind_brush_function(figs, df)

button = ph.get_reset_brush_button(figs)
dashboard1 = VBox([button, fig_a,
                  HBox([img_widget, fig_b])])


### Dashboard 2

df.sort_values(by=["experiment_id"])
traced_fig_1, dfs_1 = ph.get_figure_widget_traced(df, "scene", metrics_selector.value, "experiment_id")

traced_fig_widget_1 = go.FigureWidget(traced_fig_1)



traced_fig_1_imw_1 = Image(value=df["loaded_imgs"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))
traced_fig_1_imw_2 = Image(value=df["loaded_gts"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))

#figs, img_widget, selected_scene_df
ph.bind_hover_function2([traced_fig_widget_1], traced_fig_1_imw_1, dfs_1, img_widget_groundtruth=traced_fig_1_imw_2)


turn_the_lights_on = ph.get_dropdown_widget(["On", "Off"], label="Turn plots:", values = [True, False])

ph.bind_dropdown_switch_traces_fn(turn_the_lights_on, traced_fig_widget_1)

dashboard2 = VBox([turn_the_lights_on, traced_fig_widget_1, HBox([traced_fig_1_imw_1,traced_fig_1_imw_2])])


### Dashboard 3


traced_fig_2, dfs_2 = ph.get_figure_widget_traced(df, "experiment_id", metrics_selector.value, "scene")

traced_fig_widget_2 = go.FigureWidget(traced_fig_2)

traced_fig_2_imw_1 = Image(value=df["loaded_imgs"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))
traced_fig_2_imw_2 = Image(value=df["loaded_gts"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))



#figs, img_widget, selected_scene_df
ph.bind_hover_function2([traced_fig_widget_2], traced_fig_2_imw_1, dfs_2, img_widget_groundtruth=traced_fig_2_imw_2)

turn_the_lights_on_2 = ph.get_dropdown_widget(["On", "Off"], label="Turn plots:", values = [True, False])

ph.bind_dropdown_switch_traces_fn(turn_the_lights_on_2, traced_fig_widget_2)


dashboard3 = VBox([turn_the_lights_on_2, traced_fig_widget_2, HBox([traced_fig_2_imw_1,traced_fig_2_imw_2])])


### To aid interaction with the plots, the best results in a tabular form are displayed below

In [6]:
df.pivot_table(index = ["experiment_id", "kernel_size", "match"], values = "bad4", aggfunc=np.mean).sort_values(by="bad4").head(10)

,,,bad4
experiment_id,kernel_size,match,
plusblg_45_5x7gc_5_gs_1_alph_0,5x7,45,0.294960
plusblg_45_5x7gc_4_gs_1_alph_0,5x7,45,0.296306
plusblg_45_5x7gc_6_gs_1_alph_0,5x7,45,0.297595
plusblg_45_7x3gc_5_gs_1_alph_0,7x3,45,0.299133
plusblg_45_7x3gc_4_gs_1_alph_0,7x3,45,0.300580
plusblg_45_7x3gc_6_gs_1_alph_0,7x3,45,0.300613
plusblg_45_5x7gc_7_gs_1_alph_0,5x7,45,0.300871
plusblg_45_7x3gc_7_gs_1_alph_0,7x3,45,0.303353
plusblg_45_5x7gc_3_gs_1_alph_0,5x7,45,0.304062


In [7]:
df.pivot_table(index = "experiment_id", columns="scene", values = "bad4", aggfunc=np.min).head(10)

scene,Adirondack,ArtL,Jadeplant,Motorcycle,MotorcycleE,Piano,PianoL,Pipes,Playroom,Playtable,PlaytableP,Recycle,Shelves,Teddy,Vintage
experiment_id,,,,,,,,,,,,,,,
plusblg_30_3x3gc_1_gs_1_alph_0,0.505534,0.411218,0.399511,0.329970,0.672204,0.405237,0.661117,0.325306,0.467458,0.516415,0.420498,0.565925,0.543152,0.286253,0.638726
plusblg_30_3x3gc_1_gs_2_alph_0,0.513088,0.455534,0.383079,0.329846,0.777039,0.392016,0.691686,0.317293,0.465685,0.500984,0.416451,0.560961,0.539693,0.289469,0.637310
plusblg_30_3x3gc_2_gs_1_alph_0,0.380640,0.362703,0.363390,0.283494,0.662853,0.374126,0.616025,0.297000,0.421583,0.500632,0.386652,0.418854,0.506093,0.252712,0.599014
plusblg_30_3x3gc_2_gs_2_alph_0,0.392462,0.420911,0.359972,0.287740,0.781834,0.365790,0.668747,0.296035,0.427801,0.500361,0.385453,0.419401,0.508048,0.264058,0.602346
plusblg_30_3x3gc_3_gs_1_alph_0,0.309102,0.342880,0.352355,0.232780,0.675249,0.360269,0.613747,0.283968,0.388717,0.482550,0.358865,0.322629,0.487301,0.237052,0.554640
plusblg_30_3x3gc_3_gs_2_alph_0,0.319730,0.417034,0.359613,0.237248,0.793837,0.351927,0.668831,0.286263,0.396654,0.487390,0.363190,0.321894,0.491773,0.246511,0.559046
plusblg_30_3x5gc_1_gs_1_alph_0,0.427124,0.384413,0.415485,0.285932,0.499347,0.395627,0.628692,0.330874,0.452963,0.515420,0.387867,0.455403,0.502367,0.217181,0.618554
plusblg_30_3x5gc_1_gs_2_alph_0,0.433621,0.404309,0.392226,0.274459,0.650587,0.380368,0.646420,0.320642,0.443148,0.481738,0.377663,0.453109,0.502424,0.205106,0.608667
plusblg_30_3x5gc_2_gs_1_alph_0,0.309381,0.334918,0.378101,0.247065,0.464086,0.360701,0.546049,0.282358,0.397817,0.456721,0.347662,0.332060,0.469004,0.183907,0.571622


### Dashboard 1: Scene w.r.t. {metric} (selection plot)
<ol>
    <li>The following figure allows to use the "lasso" tool as a tool of selection.</li>
    <li>As a result, the relevant datapoints and their corresponding values in the figure in the bottom right corner will be highlighted.</li>
    <li>Pressing the "clear selection" button will reset the figure.</li>
    <li> Additionally, if a data point is hovered, the corresponding disparity output value will be displayed in the bottom right corner.</li>

</ol>

In [8]:
dashboard1

    'data': [{'custo…

### Dashboard 2: Scenes w.r.t. {metric} with color coded "epochs"
An "epoch" in this context means an experiment with the same settings evaluated across every scene in the Middlebury 2004 training dataset.<br>
<ol>
    <li>The following figure allows to turn all the plots on and off</li>
    <li>Additionally, their visibiliy can also be handled by interacting with their legend entries on the right side of the plot.
    </li>
    <li> Therefore custom comparison can be made between different scenes, kernel sizes and match values. </li>
    <li> The figure in the bottom left corner shows the corresponding disparity map. </li>
    <li> The figure in the bottom right corner shows the corresponding ground truth disparity map. </li>
</ol>

In [9]:
dashboard2

### Dashboard 3: "Epoch" w.r.t. {metric} with color coded scenes
An "epoch" in this context means an experiment with the same settings evaluated across every scene in the Middlebury 2004 training dataset.<br>
<ol>
    <li>The following figure allows to turn all the plots on and off</li>
    <li>Additionally, their visibiliy can also be handled by interacting with their legend entries on the right side of the plot.
    </li>
    <li> Therefore custom comparison can be made between different scenes, kernel sizes and match values. </li>
    <li> The figure in the bottom left corner shows the corresponding disparity map. </li>
    <li> The figure in the bottom right corner shows the corresponding ground truth disparity map. </li>
</ol>

In [10]:
dashboard3